## Example for collapsable settings

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px


In [6]:

class Figure(go.FigureWidget):
    def __init__(self, x, y, labels, layout=None, **kwargs):

        self._x = x
        self._y = y
        self._labels = labels

        self._regression_trace = None
        self._complex_hull_traces = None

        super().__init__(None, layout, **kwargs)

        # All permanent layout settings are defined here
        self.update_layout(
            hoverlabel=dict(bgcolor="white", font_size=16, font_family="Rockwell"),
            width=800,
            height=400,
            margin=dict(l=50, r=50, b=70, t=20, pad=4),
        )
        self.update_xaxes(
            ticks="outside", tickwidth=1, ticklen=10, linewidth=1, linecolor="black"
        )
        self.update_yaxes(
            ticks="outside", tickwidth=1, ticklen=10, linewidth=1, linecolor="black"
        )

        for (x, y, label) in zip(x, y, labels):
            self.add_trace(go.Scatter(x=x, y=y, name=label, mode="markers"))

    def add_regression_line(self, p1, p2):
        """
        Note: solution of the intersection of the line and the boundary box
        Arguments:
        - p1: point in 2d
        - p2: point in 2d
        """

        self._regression_trace = go.Scatter(x = [p1[0], p2[0]],y = [p1[1], p2[1]], name="Line", mode="lines")
        self.add_trace(self._regression_trace)

    def add_complex_hull(self):
        
        for (x, y, label) in zip(self._x, self._y, self._labels):
            if len(x) < 3: continue

            points = np.column_stack((x, y))
            hull = ConvexHull(points)
            
            inds = np.append(hull.vertices, hull.vertices[0])
            # TODO: use the same color as the datapoints
            self.add_trace(go.Scatter(x=x[inds], y=y[inds], name=f'{label} (hull)', mode="lines"))

            # for simplex in hull.simplices:
            #     self.add_trace(go.Scatter(x=points[simplex, 0], y=points[simplex, 1]))


In [11]:
# Generate data

N = 100

df = pd.DataFrame(np.random.randn(N, 4), columns=list("ABCD"))
df["E"] = pd.Series(np.random.randint(0, 10, size=N), dtype="category")
df["F"] = pd.Series(np.random.randint(-1, 2, size=N), dtype="category")


df

,A,B,C,D,E,F
0,-0.708460,-1.645335,1.864420,0.038818,7,1
1,0.227282,-1.222134,1.509324,-0.100534,2,0
2,0.814827,-0.449020,0.360181,0.201224,1,-1
3,0.549545,-1.153702,0.294803,1.526815,6,-1
4,0.571074,0.010946,0.099697,-0.117568,3,-1
...,...,...,...,...,...,...
95,0.593345,0.359606,1.314848,-0.213791,1,0
96,0.921754,-0.906730,0.647544,0.569625,1,0
97,-0.129910,-0.950019,-0.147889,-0.732012,7,1
98,0.589871,3.011193,-3.401698,-1.131253,4,1


In [12]:
# extract features

target = 'F'
feature_x = 'A'
feature_y = 'B'

labels = df[target].unique().tolist()

x = []
y = []
for label in labels:
    mask = df['F']==label
    x.append(df[feature_x][mask].to_numpy())
    y.append(df[feature_y][mask].to_numpy())


In [13]:

fig = Figure(x, y, labels)

In [14]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px

class Settings(GridspecLayout):
    def __init__(
        self,
        embedding_features,
        hover_features,
        feature_x,
        feature_y,
        fracture,
        **kwargs
    ):
        super().__init__(3, 3)

        widget_feature_x = widgets.Dropdown(
            description="x-axis",
            options=embedding_features,
            value=feature_x,
            layout=widgets.Layout(width="250px"),
        )

        widget_feature_y = widgets.Dropdown(
            description="y-axis",
            options=embedding_features,
            value=feature_y,
            layout=widgets.Layout(width="250px"),
        )

        widget_fracture = widgets.BoundedFloatText(
            min=0,
            max=1.0,
            # step=0.01,
            value=fracture,
            layout=widgets.Layout(left="98px", width="60px"),
        )

        widget_facture_label = widgets.Label(
            value="Fraction: ", layout=widgets.Layout(left="95px")
        )

        widget_feature_color = widgets.Dropdown(
            description="Color",
            options=["Default color"] + hover_features,
            value="Default color",
            layout=widgets.Layout(width="250px"),
        )

        widget_feature_color_type = widgets.RadioButtons(
            options=["Gradient", "Discrete"],
            value="Gradient",
            layout=widgets.Layout(width="140px", left="90px"),
        )

        widget_feature_color_list = widgets.Dropdown(
            options=px.colors.named_colorscales(),
            value="viridis",
            layout=widgets.Layout(width="65px", height="35px", left="40px"),
        )

        widget_feature_marker = widgets.Dropdown(
            description="Marker",
            options=["Default size"] + hover_features,
            value="Default size",
            layout=widgets.Layout(width="250px"),
        )
        widget_feature_marker_minvalue = widgets.BoundedFloatText(
            min=0,
            # max=self.max_value_markerfeat,
            step=1,
            # value=self.min_value_markerfeat,
            layout=widgets.Layout(left="91px", width="60px", height="10px"),
        )
        widget_feature_marker_minvalue_label = widgets.Label(
            value="Min value: ", layout=widgets.Layout(left="94px", width="70px")
        )
        widget_feature_marker_maxvalue = widgets.BoundedFloatText(
            # min=self.min_value_markerfeat,
            step=1,
            # value=self.max_value_markerfeat,
            layout=widgets.Layout(left="91px", width="60px"),
        )
        widget_feature_marker_maxvalue_label = widgets.Label(
            value="Max value: ", layout=widgets.Layout(left="94px", width="70px")
        )

        self[0, 0] = widget_feature_x
        self[1, 0] = widget_feature_y
        self[2, 0] = widgets.Box([widget_facture_label, widget_fracture])

        self[0, 1] = widget_feature_color
        self[1:, 1] = widgets.HBox(
            [widget_feature_color_type, widget_feature_color_list],
            layout=widgets.Layout(top="10px"),
        )

        self[0, 2] = widget_feature_marker
        self[1, 2] = widgets.Box(
            [
                widget_feature_marker_minvalue_label,
                widget_feature_marker_minvalue,
            ]
        )

        self[2, 2] = widgets.Box(
            [
                widget_feature_marker_maxvalue_label,
                widget_feature_marker_maxvalue,
            ]
        )

        self.layout.height = "140px"
        # self.layout.top = "30px"



embedding_features = ["A", "B", "C"]
hover_features = ["AA", "BB", "CC"]
feature_x = "A"
feature_y = "B"
fracture = 1.0

settings = Settings(
    embedding_features, hover_features, feature_x, feature_y, fracture
)
settings

Settings(children=(Dropdown(description='x-axis', layout=Layout(grid_area='widget001', width='250px'), options…

In [15]:
b1 = widgets.Button(description="|||", layout=widgets.Layout(width="40px"))

# create some control elements
int_slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description="freq")
text_xlabel = widgets.Text(value="", description="xlabel", continuous_update=False)
text_ylabel = widgets.Text(value="", description="ylabel", continuous_update=False)

text_xlabel.value = "x"
text_ylabel.value = "y"

controls = widgets.VBox([int_slider, text_xlabel, text_ylabel])

def b1_clicked(_):
    if settings.layout.display == "block" or settings.layout.display is None:
       settings.layout.display = "none"
    else:
       settings.layout.display = "block"


b1.on_click(b1_clicked)

# display(widgets.HBox([widgets.VBox([b1, controls]), fig]))
display(widgets.HBox([widgets.VBox([b1, settings]), fig]))


In [16]:
# create some control elements
int_slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description='freq')
text_xlabel = widgets.Text(value='', description='xlabel', continuous_update=False)
text_ylabel = widgets.Text(value='', description='ylabel', continuous_update=False)
 
text_xlabel.value = 'x'
text_ylabel.value = 'y'

controls = widgets.VBox([int_slider, text_xlabel, text_ylabel])

left = widgets.Accordion([controls])
left.set_title(0, '|||')


app = widgets.HBox([
   left ,
   fig,
])

app

In [17]:
class MyView(widgets.Box):
    def __init__(self):
        
        children = []
        layout = widgets.Layout(
            display="flex",
            flex_flow="column",
            border="solid 2px",
            align_items="stretch",
            # width="50%",
        )

        super().__init__(children, layout=layout)


In [18]:
w = MyView()
w

MyView(layout=Layout(align_items='stretch', border='solid 2px', display='flex', flex_flow='column'))

In [19]:
class FigureWidget:
    pass


class AtomisticViewer:
    pass


class SettingsWidget:
    pass


class Visualizer:
    def __init__(self, *args, **kwargs):

        self.figure = FigureWidget()
        self.viewer = AtomisticViewer()
        self.settings = SettingsWidget()

        # TODO: link events together between different widgets

def make(
    data: pd.DataFrame,
    embedding_features: list[str],
    hover_features: list[str],
    targets: list[str],
    smart_frac: float,
    convex_hull: bool,
    regr_line_coefs: list[float],
    path_to_structures: list[str],
):
    """
    df: pandas dataframe containing all data to be visualized
    embedding_features: list of features used for embedding
    hover_features: list of features shown while hovering
    target: feature used to create traces (same target value - same trace)
    smart_frac: fraction of points is selected to maximize visualization of data distribution
    convex_hull: convex hull is drawn around each trace
    regr_line_coefs: coeffs of a regression line
    path_to_structures: path to a directory that contains all 'xyz' structures to be visualized
    """
    pass


In [20]:
class AtomisticViewerWidget(widgets.HBox):
    pass


In [21]:
structures_list = ["h2o", "co2"]


widget_structure = widgets.Combobox(
    placeholder="",
    description="Structure:",
    options=structures_list,
    # layout=widgets.Layout(width="200px"),
)

widget_perv_button = widgets.Button(
    description="<", layout=widgets.Layout(width="50px")
)
widget_next_button = widgets.Button(
    description=">", layout=widgets.Layout(width="50px")
)

widget_label = widgets.Label('1/6', layout=widgets.Layout(width="50px", display="flex", justify_content="center"))

output = widgets.Output(layout = widgets.Layout(width="400px", height="350px"))


widgets.VBox(
    [
        widgets.HBox(
            [
                widget_structure,
                widget_perv_button,
                widget_label,
                widget_next_button
            ]
        ),
        output,
    ]
)
